In [1]:
import numpy as np 
from astropy.io import ascii

import matplotlib.pyplot as plt
import seaborn as sns

# load in isochrone package

In [2]:
import sys
sys.path.append("/Users/yaguang/anaconda/envs/isochrones/lib/python3.7/site-packages")
# from isochrones.mist import MISTEvolutionTrackGrid
# grid_tracks = MISTEvolutionTrackGrid()
# print(len(grid_tracks.df))
# grid_tracks.df.head()

# Generate the stellar properties of synthetic stars from MIST
First I need to test if this works.

In [3]:
# from isochrones import get_ichrone
# tracks = get_ichrone('mist', tracks=True)
# mass, age, feh = (1.00, 9.72, 0.0)
# tracks.generate(mass, age, feh, return_dict=True)
from isochrones import get_ichrone

from isochrones.mist import MISTEvolutionTrackGrid, MISTIsochroneGrid

track_grid = MISTEvolutionTrackGrid()

property_list=["delta_nu", "nu_max", "phase", "age", "initial_mass", "mass", "density", "Teff", "logL",'radius','logg']
#['BP_mag','G_mag','H_mag', 'J_mag','K_mag','Kepler_mag','Mbol','RP_mag','TESS_mag','Teff','W1_mag','W2_mag','W3_mag','age','delta_nu','density','dt_deep','eep','feh','initial_mass','interpolated','logL','logTeff','logg','mass','nu_max','phase','radius','star_age']
track_grid.interp([-0.12, 1.01, 353.1], property_list)

array([1.33059902e+02, 2.79813713e+03, 0.00000000e+00, 9.60364978e+00,
       1.01000000e+00, 1.00983180e+00, 1.24230972e+00, 6.03383320e+03,
       1.16066045e-01, 1.04691913e+00, 4.40266419e+00])

Read in synthetic stars.

In [4]:
import ebf
d=ebf.read('/Users/yaguang/Onedrive/Work/nike/sample/kepler_galaxia_mrtd5.ebf','/')
Nstar = d["alpha"].shape[0]
Nproperty = len(property_list)
# mist = np.zeros((Nstar, Nproperty+3))
# property_input = np.array([d["smass"], d["log_age"], d["feh"]])
# properties = track_grid.interp(property_input, property_list)
# property_output = np.concatenate([property_input.T, properties],axis=1)
# output = np.array([tuple(property_output[i,:].tolist()) for i in range(Nstar)], dtype=[(p, '>f4') for p in ["feh", "smass", "log_age"]+property_list])
# np.save("sample/mist",output)

from isochrones import get_ichrone
tracks = get_ichrone('mist')
factor = int(Nstar/16000)
idx = np.arange(0,int(Nstar/factor))*factor
mass, age, feh = d["smass"][idx], d["log_age"][idx], d["feh"][idx]
# now clean up some mess that eep couldn't be generated
trash_points = [(1.313927173614502, 9.706974029541016, 0.6776756644248962),
                (1.347928524017334, 9.659332275390625, 0.6767454743385315),
                (1.313927173614502, 9.706974029541016, 0.6776756644248962),
                (1.396459698677063, 9.623114585876465, 0.5287598967552185),
                (1.2990785837173462, 9.726263999938965, 0.5043005347251892),
                (1.5403650999069214, 9.499214172363281, 0.5627012848854065),
                (1.200117826461792, 9.819570541381836, 0.5422018766403198),
                (1.0700803995132446, 9.974210739135742, 0.5239214301109314),
                (1.0907773971557617, 9.973261833190918, 0.7108012437820435),
                (1.3396896123886108, 9.662361145019531, 0.5656616687774658),
                (1.1802692413330078, 9.860368728637695, 0.7155909538269043),
                (1.3642323017120361, 9.663329124450684, 0.5944953560829163),
                (1.2346166372299194, 9.788309097290039, 0.5223724842071533),
                (1.1053928136825562, 9.963691711425781, 0.6884390711784363),
                (1.1053928136825562, 9.963691711425781, 0.6884390711784363),
                (1.556543231010437, 9.481904983520508, 0.596089780330658),
                (1.3418062925338745, 9.66604232788086, 0.579115629196167),
                (1.0958917140960693, 9.968875885009766, 0.6464157700538635),
                (1.1296329498291016, 9.912899017333984, 0.7266412377357483),
                (1.1752325296401978, 9.869548797607422, 0.5316367149353027),
                (1.0989367961883545, 9.956917762756348, 0.5183799266815186),
                (1.1793427467346191, 9.84288215637207, 0.6021394729614258),
                (1.1360722780227661, 9.898009300231934, 0.5938041806221008)]
for point in trash_points:
    idx = (mass!=point[0]) & (age!=point[1]) & (feh!=point[2])
    mass, age, feh = mass[idx], age[idx], feh[idx]

Nstar = mass.shape[0]
eep = tracks.get_eep(mass, age, feh, accurate=True, return_nan=True)

property_input = np.array([mass,age,feh])
property_from_mist = tracks.interp_value([eep, age, feh], property_list)
property_output = np.concatenate([property_input.T, property_from_mist], axis=1)
output = np.array([tuple(property_output[i,:].tolist()) for i in range(Nstar)], dtype=[(p, '>f4') for p in ["smass", "log_age", "feh"]+property_list])
np.save("sample/mist",output)

In [11]:
point=(1.1360722780227661, 9.898009300231934, 0.5938041806221008)
np.where((mass==point[0]) & (age==point[1]) & (feh==point[2]))[0]

array([16738])